#### Dependencies

In [1]:
import pandas as pd
import numpy as np

# to install sklearn, use: pip install scikit-learn
from sklearn import metrics
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

from sklearn.preprocessing import LabelEncoder

# to install IPython, use: pip install ipython
from IPython.display import display 

from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-03-30 18:31:23.440298: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 18:31:23.620316: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-30 18:31:24.287988: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-30 18:31:25.856920: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Read Data

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv("IoMT_Flows_Sample.csv")

# Display the DataFrame
pd.set_option('display.max_columns', None)
display(df)

,proto,traffic,is_attack,total_bytes,total_pkts,pkts_unidirectional_traffic,pkt_difference,byte_difference,total_data_pkts,payload_ratio,total_payload_volume,fwd_bwd_pkts_diff,duration_weighted_pkts,pkts_size_weighted,flow_pkts_size_weighted,header_size_ratio,total_header_size,header_size_diff,fwd_bwd_payload_tot_diff,fwd_bwd_payload_avg_diff,flow_fwd_payload_diff,flow_bwd_payload_diff,flow_payload_range,iat_is_unidirectional,total_activity,history_originator,history_responder
0,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,0.000000,0.000000,0.000000,320.000000,320,320,30.0,3.0,0.0,30.0,6.0,True,1.647218e+07,1,1
1,2,normal,0,100,6,1,3,100,6,1.0,6,0.000000,6.000432,190266.749369,95133.374718,0.960000,48,0,-3.0,-1.0,103.0,100.0,3.0,False,4.063471e+02,1,1
2,1,rudeadyet,1,0,3,1,1,0,0,0.0,0,-89.221528,2.999985,0.000000,0.000000,0.492308,96,32,0.0,0.0,0.0,0.0,0.0,False,1.267669e+02,1,1
3,1,normal,0,1656,8,1,276,1656,4,0.5,4,0.000000,8.000001,15.695376,7.847688,0.992248,256,0,24.0,6.0,0.0,24.0,6.0,False,9.353475e+06,1,3
4,1,camoverflow,1,0,1,1,0,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,True,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585738,1,netscan,1,0,2,0,0,0,0,0.0,0,0.000000,2.000360,0.000000,0.000000,1.142857,44,4,0.0,0.0,0.0,0.0,0.0,True,8.645884e-03,1,1
585739,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,10.000000,14.754888,7.377444,0.993789,320,0,30.0,6.0,0.0,30.0,6.0,False,1.653697e+07,1,1
585740,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,0.000000,0.000000,0.000000,320.000000,320,320,30.0,3.0,0.0,30.0,6.0,True,1.670360e+07,1,1
585741,1,netscan,1,0,2,0,0,0,0,0.0,0,0.000000,1.998881,0.000000,0.000000,1.142857,44,4,0.0,0.0,0.0,0.0,0.0,True,5.718499e-02,1,1



### Data Pre-processing

One Hot Encoding

In [3]:
# look for columns:
## - 'proto' and 'proto_*'
## - 'pkts_unidirectional_traffic' and 'pkts_unidirectional_traffic_*'
## - 'iat_is_unidirectional' and 'iat_is_unidirectional_*'
## - 'history_responder' and 'history_responder_*'
## - 'history_originator' and 'history_originator_*'

display(df)

,proto,traffic,is_attack,total_bytes,total_pkts,pkts_unidirectional_traffic,pkt_difference,byte_difference,total_data_pkts,payload_ratio,total_payload_volume,fwd_bwd_pkts_diff,duration_weighted_pkts,pkts_size_weighted,flow_pkts_size_weighted,header_size_ratio,total_header_size,header_size_diff,fwd_bwd_payload_tot_diff,fwd_bwd_payload_avg_diff,flow_fwd_payload_diff,flow_bwd_payload_diff,flow_payload_range,iat_is_unidirectional,total_activity,history_originator,history_responder
0,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,0.000000,0.000000,0.000000,320.000000,320,320,30.0,3.0,0.0,30.0,6.0,True,1.647218e+07,1,1
1,2,normal,0,100,6,1,3,100,6,1.0,6,0.000000,6.000432,190266.749369,95133.374718,0.960000,48,0,-3.0,-1.0,103.0,100.0,3.0,False,4.063471e+02,1,1
2,1,rudeadyet,1,0,3,1,1,0,0,0.0,0,-89.221528,2.999985,0.000000,0.000000,0.492308,96,32,0.0,0.0,0.0,0.0,0.0,False,1.267669e+02,1,1
3,1,normal,0,1656,8,1,276,1656,4,0.5,4,0.000000,8.000001,15.695376,7.847688,0.992248,256,0,24.0,6.0,0.0,24.0,6.0,False,9.353475e+06,1,3
4,1,camoverflow,1,0,1,1,0,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,True,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585738,1,netscan,1,0,2,0,0,0,0,0.0,0,0.000000,2.000360,0.000000,0.000000,1.142857,44,4,0.0,0.0,0.0,0.0,0.0,True,8.645884e-03,1,1
585739,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,10.000000,14.754888,7.377444,0.993789,320,0,30.0,6.0,0.0,30.0,6.0,False,1.653697e+07,1,1
585740,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,0.000000,0.000000,0.000000,320.000000,320,320,30.0,3.0,0.0,30.0,6.0,True,1.670360e+07,1,1
585741,1,netscan,1,0,2,0,0,0,0,0.0,0,0.000000,1.998881,0.000000,0.000000,1.142857,44,4,0.0,0.0,0.0,0.0,0.0,True,5.718499e-02,1,1


In [4]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# perform one-hot encoding in columns:
df = pd.get_dummies(df, columns=['col_name'], prefix='col_name', dtype=int)

# TODO: 
# ADD CODE HERE FOR ONE_HOT ENCONDING IN COLUMNS:
### 'proto'
### 'pkts_unidirectional_traffic'
### 'iat_is_unidirectional'
### 'history_responder'
### 'history_originator'


KeyError: "None of [Index(['col_name'], dtype='object')] are in the [columns]"

In [31]:
# look for columns:
## - 'proto' and 'proto_*'
## - 'pkts_unidirectional_traffic' and 'pkts_unidirectional_traffic_*'
## - 'iat_is_unidirectional' and 'iat_is_unidirectional_*'
## - 'history_responder' and 'history_responder_*'
## - 'history_originator' and 'history_originator_*'

display(df)

,proto,traffic,is_attack,total_bytes,total_pkts,pkts_unidirectional_traffic,pkt_difference,byte_difference,total_data_pkts,payload_ratio,total_payload_volume,fwd_bwd_pkts_diff,duration_weighted_pkts,pkts_size_weighted,flow_pkts_size_weighted,header_size_ratio,total_header_size,header_size_diff,fwd_bwd_payload_tot_diff,fwd_bwd_payload_avg_diff,flow_fwd_payload_diff,flow_bwd_payload_diff,flow_payload_range,iat_is_unidirectional,total_activity,history_originator,history_responder
0,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,0.000000,0.000000,0.000000,320.000000,320,320,30.0,3.0,0.0,30.0,6.0,True,1.647218e+07,1,1
1,2,normal,0,100,6,1,3,100,6,1.0,6,0.000000,6.000432,190266.749369,95133.374718,0.960000,48,0,-3.0,-1.0,103.0,100.0,3.0,False,4.063471e+02,1,1
2,1,rudeadyet,1,0,3,1,1,0,0,0.0,0,-89.221528,2.999985,0.000000,0.000000,0.492308,96,32,0.0,0.0,0.0,0.0,0.0,False,1.267669e+02,1,1
3,1,normal,0,1656,8,1,276,1656,4,0.5,4,0.000000,8.000001,15.695376,7.847688,0.992248,256,0,24.0,6.0,0.0,24.0,6.0,False,9.353475e+06,1,3
4,1,camoverflow,1,0,1,1,0,0,0,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0.0,0.0,0.0,True,0.000000e+00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585738,1,netscan,1,0,2,0,0,0,0,0.0,0,0.000000,2.000360,0.000000,0.000000,1.142857,44,4,0.0,0.0,0.0,0.0,0.0,True,8.645884e-03,1,1
585739,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,10.000000,14.754888,7.377444,0.993789,320,0,30.0,6.0,0.0,30.0,6.0,False,1.653697e+07,1,1
585740,1,normal,0,0,10,1,0,0,5,0.5,5,0.000000,0.000000,0.000000,0.000000,320.000000,320,320,30.0,3.0,0.0,30.0,6.0,True,1.670360e+07,1,1
585741,1,netscan,1,0,2,0,0,0,0,0.0,0,0.000000,1.998881,0.000000,0.000000,1.142857,44,4,0.0,0.0,0.0,0.0,0.0,True,5.718499e-02,1,1


Normalize - Range 0 to 1

In [32]:
cols_to_normalize = [
    'payload_ratio',
    'total_payload_volume',
    'fwd_bwd_pkts_diff',
    'flow_pkts_size_weighted',
    'pkts_size_weighted',
    'duration_weighted_pkts',
    'total_bytes',
    'pkt_difference',
    'byte_difference',
    'header_size_ratio',
    'total_header_size',
    'header_size_diff',
    'fwd_bwd_payload_tot_diff',
    'fwd_bwd_payload_avg_diff',
    'flow_fwd_payload_diff',
    'flow_bwd_payload_diff',
    'flow_payload_range',
    'total_activity',
    'total_pkts',
    'total_data_pkts'
]

In [33]:
# look for statistics 
print(df[cols_to_normalize].describe())

       payload_ratio  total_payload_volume  fwd_bwd_pkts_diff  \
count  585743.000000         585743.000000       5.857430e+05   
mean        0.441087              3.463370      -1.741292e+01   
std         0.368510              4.727496       1.687960e+03   
min         0.000000              0.000000      -2.906656e+03   
25%         0.000000              0.000000       0.000000e+00   
50%         0.500000              4.000000       0.000000e+00   
75%         0.500000              5.000000       0.000000e+00   
max         1.000000             30.000000       1.048576e+06   

       flow_pkts_size_weighted  pkts_size_weighted  duration_weighted_pkts  \
count             5.857430e+05        5.857430e+05           585743.000000   
mean              1.319574e+05        2.637537e+05                4.045172   
std               2.647460e+06        5.294859e+06                5.429189   
min               0.000000e+00        0.000000e+00                0.000000   
25%               0.0000

In [ ]:
def normalize(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        else:
            # TODO:
            # COMPLETE CODE TO NORMALIZE VALUES 
            #df[col] = ...

    return df

In [ ]:
df = normalize(df, cols_to_normalize)

# look for statistics 
print(df[cols_to_normalize].describe())

Statistics on classes

In [ ]:
df.groupby('is_attack')['is_attack'].count()

In [ ]:
df.groupby('traffic')['traffic'].count()

Save Data

In [ ]:
display(df)

In [ ]:
df.to_csv('IoTMT-Sample_Processed.csv')


### Binary Classification

Removing binary and multiclass target columns from training set (i.e., x)

In [ ]:
# remove column used for multiclass classification
df = df.drop('traffic', axis=1)

# remove binary target column from x
x_columns = df.columns.drop('is_attack')
x = df[x_columns].values

print(df[x_columns])

In [ ]:
print(x)

Preparing target data

In [ ]:
y = df["is_attack"].values
print(y)

Split data into training and test sets

In [ ]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

Statistics on training data

Shape

In [ ]:
x_train.shape,y_train.shape

Occurences of each value in y_train

In [ ]:
index = pd.Index(y_train)
index.value_counts()

Percentage of each value in y_train

In [ ]:
s = pd.Series(y_train)
s.value_counts(normalize=True)

Statistics on test data

Shape

In [ ]:
x_test.shape,y_test.shape

Occurences of target each value in y_test

In [ ]:
index = pd.Index(y_test)
index.value_counts()

Percentage of each target value in y_test

In [ ]:
s = pd.Series(y_test)
s.value_counts(normalize=True)

The model

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 


Training the model

In [ ]:

# Train the model
model.fit(x_train, y_train, epochs=5)

Evaluate the model

In [ ]:
pred = model.predict(x_test)
print(pred)

In [ ]:
pred = np.round(pred).astype(int)
print(pred)

Confusion Matrix

In [ ]:
conf_mat = metrics.confusion_matrix(y_test, pred)
conf_mat

In [ ]:
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = ["Normal", "Attack"])
 
# display matrix
cm_display.plot()
plt.show()

Metrics

* **Accuracy** $\large = \frac{TP ~+~ TN}{TP ~+~ TN ~+~ FP ~+~ FN} ~=~ \frac{TP ~+~ TN}{\textrm{Number of Predictions} }$ 

* **Precision** $\large = \frac{TP}{TP ~+~ FP} ~=~ \frac{TP}{\textrm{Number of Positive Predictions} }$ 

* **Recall** $\large =\frac{TP}{TP ~+~ FN} ~=~ \frac{TP}{\textrm{Number of Actual Positive Observations} }$

* **F1** $\large =\frac{2 ~*~ Precision ~*~ Recall}{Precision ~+~ Recall}$



In [ ]:

# Calculates performance metrics
acc = accuracy_score(y_true =  y_test, y_pred = pred)
print(f'Accuracy : {np.round(acc*100,2)}%')
          
precision = precision_score(y_true =  y_test, y_pred = pred)
print(f'Precision : {np.round(precision*100,2)}%')
              
recall = recall_score(y_true =  y_test, y_pred = pred)
print(f'Recall: {np.round(recall*100,2)}%')

f1 = f1_score(y_true =  y_test, y_pred = pred)
print(f'F1-score: {np.round(f1*100,2)}%')



### Multiclass Classification

Load pre-processed data

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv("IoTMT-Sample_Processed.csv", index_col=0)

# Display the DataFrame
pd.set_option('display.max_columns', None)
display(df)

Train model 

Removing binary and multiclass target columns from training set (i.e., x)

In [ ]:
# remove column used for binary classification
df = df.drop('is_attack', axis=1)

# remove target column 
x_columns = df.columns.drop('traffic')

x = df[x_columns].values

print(df[x_columns])

Encoding and preparing target values

In [ ]:

le = LabelEncoder()
le.fit(df["traffic"].values)

y = df["traffic"].values
y = le.transform(y)

print(y)

Verifying encoding

In [ ]:
print(le.classes_)

In [ ]:
print(le.transform(le.classes_))

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

Splitting data into training and test sets

In [ ]:
# Split into training and testing sets

# TODO:
# Create the training and test sets

Statistincs on training data

Shape of training data

In [ ]:
x_train.shape,y_train.shape

Occurrences of each target value in training data

In [ ]:
index = pd.Index(y_train)
index.value_counts()

Percentage of each target value in training data

In [ ]:
s = pd.Series(y_train)
s.value_counts(normalize=True)

Statistics on test data

Shape of test data

In [ ]:
x_test.shape,y_test.shape

Occurrences of each target value in test data

In [ ]:
index = pd.Index(y_test)
index.value_counts()

Percentage of each target value in training data

In [ ]:
s = pd.Series(y_test)
s.value_counts(normalize=True)

The model

In [ ]:
# TODO:

# Define and compile de model for multiclass classification
# Attention to the number of neurons in the output layer 

Training the model

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=5)

Evaluate the model

In [ ]:
pred = model.predict(x_test)
print(pred)


In [ ]:
pred = np.round(pred).astype(int)
print(pred)

In [ ]:
pred = np.argmax(pred, axis=1)
print(pred)

Confusion Matrix

In [ ]:
conf_mat = metrics.confusion_matrix(y_test, pred)
conf_mat

In [ ]:

clabels = ['apachekiller', 'arpspoofing', 'camoverflow', 'mqttmalaria', 'netscan', 'normal', 'rudeadyet', 'slowloris', 'slowread']

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = clabels)
 
# display matrix
cm_display.plot()
plt.show()


Performance metrics

- Accuracy = $\large = \frac{Correct~Predictions}{All~Predictions}$
- Precision for a given class = $\large = \frac{Correct~Predictions~for~the~Class}{All~Predictions~for~the~Class}$   
- Recall for a given class = $\large = \frac{Correct~Predictions~for~the~Class}{All~Instances~of~the~Class}$  

- Averaging is a way to get a single number for multiclass. Depending on the importance one wants to give to minority classes: 
    - Macro average: Compute the metric for each class, and returns the average without considering the proportion for each class in the dataset. For instance:

        Precision = $\large = \frac{P_{class 1} ~+~ P_{class 2} ~+~ ... ~+~ P_{class n}}{N}$   

    
    - Weighted average: Compute the metric for each class, and returns the average considering the proportion (weighted) for each class in the dataset. For instance:

        Precision = $\large = \frac{N_1 ~*~ P_{class 1} ~+~ N_2 ~*~ P_{class 2} ~+~ ... ~+~ N_n ~*~ P_{class n}}{N}$   


 

In [ ]:

# Calculates performance metrics
acc = accuracy_score(y_true =  y_test, y_pred = pred)
print(f'Accuracy : {np.round(acc*100,2)}%')
          
precision = precision_score(y_true =  y_test, y_pred = pred, average='macro')
print(f'Precision - Macro: {np.round(precision*100,2)}%')

# TODO:
## compute recall and f1 score using macro average
## compute precision and recall using weighted average


f1 = f1_score(y_true =  y_test, y_pred = pred, average='weighted')
print(f'F1-score - Weighted: {np.round(f1*100,2)}%')
